In [ ]:
import hopsworks
import pandas as pd
import tensorflow as tf
from keras.layers import Dense
from keras.layers import LSTM
from keras.models import Sequential
import os
import numpy as np
import joblib
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# # You have to set the environment variable 'HOPSWORKS_API_KEY' for login to succeed
project = hopsworks.login()
fs = project.get_feature_store()

In [ ]:
# Get features (data) from feature store
# gold data
df_gold = pd.read_csv("Tools/GC=F.csv")
print(df_gold)
y_gold = df_gold['Close'].values.reshape(-1,1)

In [ ]:
# forecast setting
n_forecast = 90  # length of output sequences (forecast period)
n_lookback = 3*n_forecast  # length of input sequences (lookback period, should be 3 times longer than forecast period)

In [ ]:
# Model Training for Gold
scaler_gold = MinMaxScaler(feature_range=(0, 1))
scaler_gold = scaler_gold.fit(y_gold)
y_gold = scaler_gold.transform(y_gold)

X_gold = []
Y_gold = []

for i in range(n_lookback, len(y_gold) - n_forecast + 1):
    X_gold.append(y_gold[i - n_lookback: i])
    Y_gold.append(y_gold[i: i + n_forecast])

X_gold = np.array(X_gold)
Y_gold = np.array(Y_gold)
print(X_gold.shape, Y_gold.shape)

# fit / train the model
model_gold = Sequential()
model_gold.add(LSTM(units=128, return_sequences=True, input_shape=(n_lookback, 1)))
model_gold.add(LSTM(units=64, return_sequences=True))
model_gold.add(LSTM(units=64, return_sequences=False))
model_gold.add(Dense(n_forecast))
model_gold.compile(loss='mean_squared_error', optimizer='adam')
model_gold.fit(X_gold, Y_gold, epochs=20, batch_size=32, verbose=1)

In [ ]:
# We will now upload our model to the Hopsworks Model Registry. First get an object for the model registry.
mr = project.get_model_registry()
model_dir="gold_model"
if os.path.isdir(model_dir) == False:
    os.mkdir(model_dir)

# Save both our model and the confusion matrix to 'model_dir', whose contents will be uploaded to the model registry
joblib.dump(model_gold, model_dir + "/gold_model.pkl")
# fig.savefig(model_dir + "/wine_confusion_matrix.png")

# # Create an entry in the model registry that includes the model's name, desc, metrics
gold_model = mr.python.create_model(
    name="gold_model",
    # metrics={"accuracy" : metrics['accuracy']},
    # model_schema=model_schema,
    description="Gold Predictor"
)

# Upload the model to the model registry, including all files in 'model_dir'
gold_model.save(model_dir)
